# Visualizing a single CT scan in python


Arguably, understanding the data we are presented should be the very first step in appreciating the problem at hand. I have decided to play around with files and visualizations before, for the sake of feeling alittle but of achievment, and getting a gut-instinct of what we have to deal with. I will turn my attention to perform more analytical study of a single CT scan, and my aim is to understand the format. I will also use this opportunity to build any additional utilities needed for future Model definition work.

I am very intrigued to visualize 1 CT scan in 3D. So far, we have visualized only cross-segments and projections. Let's spend some time to look up resources that will help us achieve visualizing in 3D.

- I came across a fantastic blog that will serve as the foundation for our analysis: [Vincente Rodriguez blog](https://vincentblog.xyz/posts/medical-images-in-python-computed-tomography).
- The [Wikipedia page](https://en.wikipedia.org/wiki/Marching_cubes) about `Marching cubes` is also a great introduction for 3D visualization.

In [1]:
from LUNA16.utils.mermaid import mm

## Understanding CT scans and Voxels

Before we get too far into the project, we need to take a moment to explain what a CT scan is. We have already started exploring the contents of our CT scans and as we progress we will be using data from them extensively for this project, so it is important to have a working understanding of the format we are dealing with. The key aspect we have to know is that CT scans are effectively 3D X-rays, represented as 3D arrays of single-channel data that is concatenated. It is like a stacked array of grayscale images.

A [voxel](https://en.wikipedia.org/wiki/Voxel) is a 3 dimensional representation of some value in 3D space.As with pixels in a 2D bitmap, *voxels themselves do not typically have their coordinates explicitly encoded with their values* Instead, they depend on a rendering systems to infer the position of a voxel, and we will revisit this point in a bit.

A Voxel is the 3D equivalent of a Pixel. As such, remember that:

- A Voxel does not have "width" encoded in it, yet it represents a volume - just as how a Pixel does not have a width value, until it is displayed by a renderer on either a screen or transformed to be printed. What this effectively means is that when you will slice a single Voxel from an array, you will retrieve a single point in memory that represents a volumetric point in 3D
- Voxels depend on some form of rendering system to infer their positions.

Voxels can either be cubic or they can have different shapes. To appreciate further more the data that constitutes this challenge, I provide few notes retrieved from the [LUNA 16 Challenge page](https://luna16.grand-challenge.org/data/):

|#|Descriptor | Note |
|-|-----------|------|
|1| The data used is from the [LIDC/IDRI database](http://wiki.cancerimagingarchive.net/display/Public/LIDC-IDRI) | What's this database? we need to research this. |
|2| Scans with slice thickness greater than 2.5 mm were excluded | Why were they excluded - is that something that will affect our work?|
|3| Radiologist have annotated the data: they have marked 'lesions' as non-nodules, <3 mm nodules, and >3 mm ones | What's a lesion, and how have they annotated it? Read their [publication](http://www.ncbi.nlm.nih.gov/pubmed/21452728). Quote "Each radiologist independently reviewed each CT scan and marked lesions belonging to one of three categories ("nodule > or =3 mm," "nodule <3 mm," and "non-nodule > or =3 mm")"|
|4| Annotations | Each finding is on a line of annotations.csv. Each line contains SeriesUID and x,y,z position in **world coordinates**, and corresponding diameter in **mm**. Do we need to perform any transformation to coordinates?
|5| Objective | The LUNA16 challenge will focus on a large-scale evaluation of *automatic nodule detection* algorithms. There are two "tracks": (1) Complete systems for nodule detection, (2) systems that use a list of locations of possible nodules. The organizers provide this list to also allow teams to participate with an algorithm that only determines the likelihood for a given location in a CT scan to contain a pulmonary nodule.|

Some links that I have found valuable are:
- [Definition of Lesions](https://en.wikipedia.org/wiki/Lesion) on Wikipedia
- [Definition of Nodules](https://en.wikipedia.org/wiki/Nodule_(medicine)) on Wikipedia
- The [History of CT scans](https://www.youtube.com/watch?v=M6vsBcxHPZU&t=785s) on Youtube


In [2]:
mm.graph(mm.data_explanation_06)

In [3]:
#%pip install ipyvolume

In [4]:
from LUNA16.utils.analyze_folders import analyze_folder
from LUNA16.utils.analyze_data_distribution import read_mhd
import matplotlib.pyplot as plt
import random
import dask
import dask.array as da
import numpy as np
from dask.distributed import Client
import SimpleITK as sitk

In [5]:
ROOT_FOLDER = "/home/azureuser/cloudfiles/data/LUNA16/extracted"
all_files = analyze_folder(ROOT_FOLDER)
assert len(all_files) == 3567
all_mhd_files = [file for file in all_files if file.extension == "mhd"]
assert len(all_mhd_files) == 1776
random_uid = random.choice([file.filename for file in all_files if file.extension =="mhd"])
notebook_files = [file for file in all_files if file.filename ==random_uid]
mhd = [file for file in notebook_files if file.extension == "mhd"]
mhd_image = sitk.ReadImage(mhd[0].folder)
mhd_image = np.array(sitk.GetArrayFromImage(mhd_image), dtype=np.float32)

In [6]:
new_V = np.clip(mhd_image, -500, 500)
middle = np.round(new_V.shape[0] /2)
middle = int(middle)

small_V = new_V[middle -1: middle + 2, 254:257, 254:257]

In [7]:
import numpy as np
import ipyvolume as ipv
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0

In [8]:
ipv.figure()
ipv.volshow(V, level=[0.25, 0.75], opacity=0.53, level_width=0.1, data_min=0, data_max=1)
ipv.view(-30, 40)
ipv.show()

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)


In [9]:
import ipyvolume as ipv
fig = ipv.figure()
vol_head = ipv.examples.head(max_shape=128)
vol_head.ray_steps = 400
ipv.view(90, 0)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ipyvolume/widgets.py:241: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_view = self.data_original[view]
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ipyvolume/utils.py:208: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = (data[slices1] + data[slices2]) / 2


(90, 0, 2.0)